场景1、入门使用

In [ ]:
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory()#注意这里return_messages默认为false
#将消息保存到内存中
#不管inputs、outputs的key用什么名字，都认为inputs的key是human，outputs的key是AI
memory.save_context(inputs={"input":"你好，我是人类"},outputs={"outputs":"你好，我是ai助手"})
memory.save_context(inputs={"input":"很开心认识你"},outputs={"outputs":"我也是"})
# print(memory.load_memory_variables())
# 缺少大括号会报错#TypeError: load_memory_variables() missing 1 required positional argument: 'inputs'
print(memory.load_memory_variables({}))

{'history': 'Human: 你好，我是人类\nAI: 你好，我是ai助手\nHuman: 很开心认识你\nAI: 我也是'}


场景2：结合chain

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
import os
import dotenv
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

template = """
你可以与人类对话，
历史：{history}，
当前问题：{question}，
"""

prompt = ChatPromptTemplate.from_template(template)

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt, memory=memory)
result=chain.invoke({"question": "你好,我叫Tom"})
print(result)

{'question': '你好,我叫Tom', 'history': '', 'text': '你好，Tom！很高兴和你聊天。有什么我可以帮助你的吗？'}


注意：上面的提示词模板中，{history}是必须的，否则会报错。因为memory默认会将历史记录存储在{history}中。由于memory的history为空，所以打印结果history为空(一对空的单引号)。

In [8]:
res=chain.invoke({"question": "我的名字是什么？"})
print(res)

{'question': '我的名字是什么？', 'history': 'Human: 你好,我叫Tom\nAI: 你好，Tom！很高兴和你聊天。有什么我可以帮助你的吗？', 'text': '你的名字是Tom。很高兴认识你！有什么我可以帮助你的吗？'}


可以通过memory_key修改memory数据的变量名，默认是history。

In [13]:
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory 
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
import os
import dotenv
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
model = ChatOpenAI(model="gpt-4o-mini")

template="""
你可以与用户进行对话。
历史：{chat_history}
用户问题：{question}
"""

prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat_history")
chain = LLMChain(llm=model, prompt=prompt, memory=memory)

res=chain.invoke({"question": "我叫Tom"})
print(res)

res=chain.invoke({"question": "我的名字是什么？"})
print(res)








{'question': '我叫Tom', 'chat_history': '', 'text': '你好，Tom！很高兴认识你。有什么我可以帮助你的吗？'}
{'question': '我的名字是什么？', 'chat_history': 'Human: 我叫Tom\nAI: 你好，Tom！很高兴认识你。有什么我可以帮助你的吗？', 'text': '你的名字是Tom。很高兴再次见到你，Tom！有什么我可以帮你的吗？'}


使用ChatPromptTemplate和return_messages=True

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
import os
import dotenv
dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
model = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

memory = ConversationBufferMemory(memory_key="history", return_messages=True)

chain = LLMChain(llm=model, prompt=prompt, memory=memory)
result=chain.invoke({"question": "中国的首都在哪里？"})
print(result)

result=chain.invoke({"question": "我刚刚问了什么问题？"})
print(result)


{'question': '中国的首都在哪里？', 'history': [HumanMessage(content='中国的首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都位于北京。', additional_kwargs={}, response_metadata={})], 'text': '中国的首都位于北京。'}
{'question': '我刚刚问了什么问题？', 'history': [HumanMessage(content='中国的首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都位于北京。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚刚问了什么问题？', additional_kwargs={}, response_metadata={}), AIMessage(content='您刚刚问了“中国的首都在哪里？”这个问题。', additional_kwargs={}, response_metadata={})], 'text': '您刚刚问了“中国的首都在哪里？”这个问题。'}
